In [1]:
from sklearn.model_selection import StratifiedKFold
import json
import numpy as np
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix
)

In [2]:
print("="*70)
print("NEURAL NETWORK - 5-FOLD CROSS VALIDATION")
print("="*70)

# ==========================
# LOAD DATA
# ==========================
data = np.load("brain_mri_preprocessed.npz")

X_train_flat = data["X_train"]
y_train      = data["y_train"]

X_test_flat  = data["X_test"]
y_test       = data["y_test"]

print("Loaded:")
print("Train:", X_train_flat.shape)
print("Test:",  X_test_flat.shape)

NEURAL NETWORK - 5-FOLD CROSS VALIDATION
Loaded:
Train: (5521, 16384)
Test: (1205, 16384)


In [3]:
# =====================================
# ACTIVATION FUNCTIONS
# =====================================
def sigmoid(z):
    return 1 / (1 + np.exp(-np.clip(z, -500, 500)))

def sigmoid_grad(z):
    s = sigmoid(z)
    return s * (1 - s)

In [4]:
# =====================================
# WEIGHT INITIALIZATION
# =====================================
def initialize_weights(input_size, hidden_size):
    epsilon = np.sqrt(6) / np.sqrt(input_size + hidden_size)
    theta1 = np.random.uniform(-epsilon, epsilon, (hidden_size, input_size + 1))
    theta2 = np.random.uniform(-epsilon, epsilon, (1, hidden_size + 1))
    return theta1, theta2

In [5]:
#=====================================
# COST FUNCTION
# =====================================
def compute_cost(X, y, theta1, theta2, lambda_reg):
    m = X.shape[0]

    # Forward
    a1 = np.hstack([np.ones((m,1)), X])
    z2 = a1 @ theta1.T
    a2 = sigmoid(z2)
    a2 = np.hstack([np.ones((m,1)), a2])
    z3 = a2 @ theta2.T
    h  = sigmoid(z3)

    h = np.clip(h, 1e-12, 1-1e-12)

    # Cost
    cost = (-1/m) * np.sum(
        y.reshape(-1,1)*np.log(h) + (1 - y.reshape(-1,1))*np.log(1-h)
    )

    # Regularization
    reg = (lambda_reg/(2*m)) * (
        np.sum(theta1[:,1:]**2) + np.sum(theta2[:,1:]**2)
    )

    return cost + reg

In [6]:
# =====================================
# PREDICT FUNCTION
# =====================================
def predict_nn(X, theta1, theta2):
    m = X.shape[0]
    a1 = np.hstack([np.ones((m, 1)), X])
    z2 = a1 @ theta1.T
    a2 = sigmoid(z2)
    a2 = np.hstack([np.ones((m, 1)), a2])
    z3 = a2 @ theta2.T
    h  = sigmoid(z3)
    return h

In [7]:
# =====================================
# TRAIN NEURAL NETWORK (FIXED BACKPROP)
# =====================================
def train_nn(X_train, y_train, hidden_units=64, alpha=0.5, 
             lambda_reg=0.1, num_iterations=400, verbose=False):

    m, n = X_train.shape
    theta1, theta2 = initialize_weights(n, hidden_units)

    for iteration in range(num_iterations):

        # Forward pass
        a1 = np.hstack([np.ones((m,1)), X_train])
        z2 = a1 @ theta1.T
        a2 = sigmoid(z2)
        a2_bias = np.hstack([np.ones((m,1)), a2])
        z3 = a2_bias @ theta2.T
        h  = sigmoid(z3)

        # Backpropagation (corrected)
        delta3 = h - y_train.reshape(-1,1)
        delta2 = (delta3 @ theta2[:,1:]) * sigmoid_grad(z2)

        # Accumulate gradients
        theta2_grad = (1/m) * (delta3.T @ a2_bias)
        theta1_grad = (1/m) * (delta2.T @ a1)

        # Regularization (excluding bias)
        theta2_grad[:,1:] += (lambda_reg/m) * theta2[:,1:]
        theta1_grad[:,1:] += (lambda_reg/m) * theta1[:,1:]

        # Gradient descent
        theta2 -= alpha * theta2_grad
        theta1 -= alpha * theta1_grad

        if verbose and iteration % 100 == 0:
            cost = compute_cost(X_train, y_train, theta1, theta2, lambda_reg)
            print(f"  Iter {iteration} - Cost: {cost:.4f}")

    return theta1, theta2

In [8]:
# =====================================
# ==========  K-FOLD CV  ==============
# =====================================
print("\n[1/3] Running 5-Fold Cross Validation...\n")

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
cv_scores_nn = []

for fold_idx, (train_idx, val_idx) in enumerate(skf.split(X_train_flat, y_train), 1):
    print(f"Fold {fold_idx}/5...")

    X_tr = X_train_flat[train_idx]
    y_tr = y_train[train_idx]
    X_vl = X_train_flat[val_idx]
    y_vl = y_train[val_idx]

    theta1, theta2 = train_nn(
        X_tr, y_tr,
        hidden_units=64,
        alpha=0.5,
        lambda_reg=0.1,
        num_iterations=400,
        verbose=False
    )

    # Predict
    y_pred = (predict_nn(X_vl, theta1, theta2) > 0.5).astype(int).flatten()
    acc = accuracy_score(y_vl, y_pred)
    cv_scores_nn.append(acc)

    print(f"  Fold {fold_idx} Accuracy: {acc:.4f}")

cv_scores_nn = np.array(cv_scores_nn)

print(f"\n5-Fold CV Summary:")
print(f"  Mean:    {cv_scores_nn.mean():.4f}")
print(f"  Std Dev: {cv_scores_nn.std():.4f}")
print(f"  Min:     {cv_scores_nn.min():.4f}")
print(f"  Max:     {cv_scores_nn.max():.4f}")




[1/3] Running 5-Fold Cross Validation...

Fold 1/5...
  Fold 1 Accuracy: 0.7421
Fold 2/5...
  Fold 2 Accuracy: 0.7681
Fold 3/5...
  Fold 3 Accuracy: 0.7428
Fold 4/5...
  Fold 4 Accuracy: 0.7428
Fold 5/5...
  Fold 5 Accuracy: 0.7545

5-Fold CV Summary:
  Mean:    0.7500
  Std Dev: 0.0102
  Min:     0.7421
  Max:     0.7681


In [9]:
# =====================================
# TRAIN FULL MODEL
# =====================================
print("\n[2/3] Training on full training set...")

theta1_final, theta2_final = train_nn(
    X_train_flat, y_train, 
    hidden_units=64, alpha=0.5, lambda_reg=0.1, 
    num_iterations=400, verbose=True
)

print("✓ Model trained")


[2/3] Training on full training set...
  Iter 0 - Cost: 0.6600
  Iter 100 - Cost: 0.5421
  Iter 200 - Cost: 0.5796
  Iter 300 - Cost: 0.5326
✓ Model trained


In [10]:
# =====================================
# TEST SET EVALUATION
# =====================================
print("\n[3/3] Evaluating on test set...")

y_test_pred_nn = (predict_nn(X_test_flat, theta1_final, theta2_final) > 0.5).astype(int).flatten()

test_acc_nn = accuracy_score(y_test, y_test_pred_nn)
test_prec_nn = precision_score(y_test, y_test_pred_nn)
test_rec_nn = recall_score(y_test, y_test_pred_nn)
test_f1_nn = f1_score(y_test, y_test_pred_nn)

print(f"\nTest Set Performance:")
print(f"  Accuracy:  {test_acc_nn:.4f}")
print(f"  Precision: {test_prec_nn:.4f}")
print(f"  Recall:    {test_rec_nn:.4f}")
print(f"  F1-score:  {test_f1_nn:.4f}")


[3/3] Evaluating on test set...

Test Set Performance:
  Accuracy:  0.7436
  Precision: 0.7436
  Recall:    1.0000
  F1-score:  0.8529


In [11]:
# Overfitting check
y_train_pred = (predict_nn(X_train_flat, theta1_final, theta2_final) > 0.5).astype(int).flatten()
train_acc_nn = accuracy_score(y_train, y_train_pred)
gap_nn = train_acc_nn - test_acc_nn

print(f"\nOverfitting Analysis:")
print(f"  Train Accuracy: {train_acc_nn:.4f}")
print(f"  Test Accuracy:  {test_acc_nn:.4f}")
print(f"  Gap:            {gap_nn:.4f} ({gap_nn*100:.2f}%)")


Overfitting Analysis:
  Train Accuracy: 0.7424
  Test Accuracy:  0.7436
  Gap:            -0.0011 (-0.11%)


In [12]:
# =====================================
# SAVE RESULTS
# =====================================
nn_results = {
    'model': 'Neural Network',
    'architecture': {
        'input_size': int(X_train_flat.shape[1]),
        'hidden_units': 64,
        'output_size': 1
    },
    'hyperparameters': {
        'learning_rate': 0.5,
        'lambda': 0.1,
        'iterations': 400
    },
    'cross_validation': {
        'mean_accuracy': float(cv_scores_nn.mean()),
        'std_dev': float(cv_scores_nn.std()),
        'fold_scores': cv_scores_nn.tolist()
    },
    'test_performance': {
        'accuracy': float(test_acc_nn),
        'precision': float(test_prec_nn),
        'recall': float(test_rec_nn),
        'f1_score': float(test_f1_nn)
    },
    'overfitting': {
        'train_accuracy': float(train_acc_nn),
        'test_accuracy': float(test_acc_nn),
        'gap': float(gap_nn)
    }
}

with open('nn_cv_results.json', 'w') as f:
    json.dump(nn_results, f, indent=4)

print("\n✓ Results saved to: nn_cv_results.json")
print("="*70)
print("NEURAL NETWORK - COMPLETED ✓")
print("="*70)


✓ Results saved to: nn_cv_results.json
NEURAL NETWORK - COMPLETED ✓
